In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical

train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

/Users/Gregoire/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
Train=train.as_matrix()
y_train=Train[:,0]
x_train=Train[:,1:]
y_train = to_categorical(y_train,5)


In [3]:
#define a validation set
split_size = int(x_train.shape[0]*0.8)

train_x, val_x = x_train[:split_size], x_train[split_size:]
train_y, val_y = y_train[:split_size], y_train[split_size:]

In [4]:
def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]].reshape(-1, input_num_units)
    
    if dataset_name == 'train':
        batch_y = eval(dataset_name + '_y')[[batch_mask]]
#         batch_y = to_categorical(batch_y, num_classes=5)
        
    return batch_x, batch_y

In [61]:
# number of neurons in each layer
input_num_units = 100 #nb of dimensions of the data
hidden_num_units = 500
hidden_num_units2 = 150
hidden_num_units3 = 100

output_num_units = 5 #nb of classes

#define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])
# set remaining variables
training_epochs = 100
batch_size = 300
learning_rate = 0.001
mom = 0.8

### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'h1': tf.Variable(tf.random_normal([input_num_units, hidden_num_units], seed=seed)),
    'h2': tf.Variable(tf.random_normal([hidden_num_units, hidden_num_units2], seed=seed)),
#     'h3': tf.Variable(tf.random_normal([hidden_num_units2, hidden_num_units3], seed=seed)),
#     'h4': tf.Variable(tf.random_normal([hidden_num_units, hidden_num_units], seed=seed)),
    'out': tf.Variable(tf.random_normal([hidden_num_units2, output_num_units], seed=seed))
}

biases = {
    'h1': tf.Variable(tf.random_normal([hidden_num_units], seed=seed)),
    'h2': tf.Variable(tf.random_normal([hidden_num_units2], seed=seed)),
#     'h3': tf.Variable(tf.random_normal([hidden_num_units3], seed=seed)),
#     'h4': tf.Variable(tf.random_normal([hidden_num_units], seed=seed)),
    'out': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

keep_prob = tf.placeholder("float")

In [62]:
#definition of the neural net computational graph
def NeuralNetDeep(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['h1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['h2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2, keep_prob)
#     layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['h3'])
#     layer_3 = tf.nn.relu(layer_3)
#     layer_3 = tf.nn.dropout(layer_3, keep_prob)
#     layer_4 = tf.add(tf.matmul(layer_3, weights['h3']), biases['h4'])
#     layer_4 = tf.nn.relu(layer_4)
#     layer_4 = tf.nn.dropout(layer_4, keep_prob)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

predictions = NeuralNetDeep(x, weights, biases, keep_prob)
#cost function
beta=0.001
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=predictions))
regularizer = tf.nn.l2_loss(weights['h1'])
regularizer += tf.nn.l2_loss(weights['h2'])
# regularizer += tf.nn.l2_loss(weights['h3'])
regularizer += tf.nn.l2_loss(weights['out'])
cost = tf.reduce_mean(cost + beta * regularizer)

#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [63]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        for i in range(total_batch):
            batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % 1 == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: val_x, y: val_y, keep_prob: 1.0}))
    print(x.shape,y.shape)
    res=sess.run(predictions,{x: test, keep_prob: 1.0})


('Epoch:', '0001', 'cost=', '391.296293852')
('Epoch:', '0002', 'cost=', '100.029109677')
('Epoch:', '0003', 'cost=', '58.262847774')
('Epoch:', '0004', 'cost=', '50.828200839')
('Epoch:', '0005', 'cost=', '47.214035868')
('Epoch:', '0006', 'cost=', '44.578770265')
('Epoch:', '0007', 'cost=', '42.508264908')
('Epoch:', '0008', 'cost=', '40.785834811')
('Epoch:', '0009', 'cost=', '39.362629720')
('Epoch:', '0010', 'cost=', '38.152754373')
('Epoch:', '0011', 'cost=', '37.084663265')
('Epoch:', '0012', 'cost=', '36.139499007')
('Epoch:', '0013', 'cost=', '35.275886485')
('Epoch:', '0014', 'cost=', '34.499978249')
('Epoch:', '0015', 'cost=', '33.786148551')
('Epoch:', '0016', 'cost=', '33.102248186')
('Epoch:', '0017', 'cost=', '32.454754912')
('Epoch:', '0018', 'cost=', '31.838868400')
('Epoch:', '0019', 'cost=', '31.222747550')
('Epoch:', '0020', 'cost=', '30.638034783')
('Epoch:', '0021', 'cost=', '30.054750203')
('Epoch:', '0022', 'cost=', '29.470630797')
('Epoch:', '0023', 'cost=', '2

In [80]:
#Extracting the results
tmp = np.argmax(res,axis=1)
dff = pd.DataFrame(tmp)
dff.reset_index(inplace=True)
dff.columns = ['Id','y']
dff['Id']+=45324
# dff.columns = ['Id','y']
dff.to_csv("outputtask3.csv", header=True, index=False)